In [20]:
import sys
import os



import numpy as np
from SynthTempNetwork import Individual, SynthTempNetwork
from TemporalNetwork import ContTempNetwork, StaticTempNetwork
from FlowStability import FlowIntegralClustering
import pickle

import matplotlib.pyplot as plt
import matplotlib

from scipy.sparse import (lil_matrix, dok_matrix, diags, eye, isspmatrix_csr, isspmatrix,
                          csr_matrix, coo_matrix, csc_matrix)
from scipy.sparse.linalg import eigsh 

In [21]:
t_start = 0
deltat1 =100
t_end = 3*deltat1

p27 = 49/50
p9 = 49/(3*50)
p3 = 49/(9*50)






def make_step_block_probs(deltat1, p27=0, p9=0, p3=0):
    """ Returns a function that generate the block probability matrix as a function of time.
    
        `deltat1` is the length of the within step
    
        `deltat2` is the length of the echanging step
        
        `m1` is the prob of self-interaction (during deltat1)
        
        `p1` is the prob of cross-interaction (during deltat2)
    """


    def block_mod_func(t):
        k27 = (1/p27 -1)/26
        pout27 = k27 * p27

        k9 = (1/p9 - 3)/24
        pout9 = k9 * p9

        k3 = (1/p3 - 9)/18
        pout3 = k3 * p3
        
        net27 = np.array([[p27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27],
                         [pout27, p27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27],
                         [pout27, pout27, p27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27],
                         [pout27, pout27, pout27, p27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27],
                         [pout27, pout27, pout27, pout27, p27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27],
                         [pout27, pout27, pout27, pout27, pout27, p27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27],
                         [pout27, pout27, pout27, pout27, pout27, pout27, p27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27],
                         [pout27, pout27, pout27, pout27, pout27, pout27, pout27, p27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27],
                         [pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, p27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27],
                         [pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, p27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27],
                         [pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, p27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27],
                         [pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, p27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27],
                         [pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, p27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27],
                         [pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, p27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27],
                         [pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, p27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27],
                         [pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, p27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27],
                         [pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, p27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27],
                         [pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, p27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27],
                         [pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, p27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27],
                         [pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, p27, pout27, pout27, pout27, pout27, pout27, pout27, pout27],
                         [pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, p27, pout27, pout27, pout27, pout27, pout27, pout27],
                         [pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, p27, pout27, pout27, pout27, pout27, pout27],
                         [pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, p27, pout27, pout27, pout27, pout27],
                         [pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, p27, pout27, pout27, pout27],
                         [pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, p27, pout27, pout27],
                         [pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, p27, pout27],
                         [pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, pout27, p27]])
        net9 = np.array([[p9, p9, p9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9],
                         [p9, p9, p9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9],
                         [p9, p9, p9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9],
                         [pout9, pout9, pout9, p9, p9, p9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9],
                         [pout9, pout9, pout9, p9, p9, p9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9],
                         [pout9, pout9, pout9, p9, p9, p9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9],
                         [pout9, pout9, pout9, pout9, pout9, pout9, p9, p9, p9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9],
                         [pout9, pout9, pout9, pout9, pout9, pout9, p9, p9, p9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9],
                         [pout9, pout9, pout9, pout9, pout9, pout9, p9, p9, p9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9],
                         [pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, p9, p9, p9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9],
                         [pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, p9, p9, p9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9],
                         [pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, p9, p9, p9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9],
                         [pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, p9, p9, p9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9],
                         [pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, p9, p9, p9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9],
                         [pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, p9, p9, p9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9],
                         [pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, p9, p9, p9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9],
                         [pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, p9, p9, p9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9],
                         [pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, p9, p9, p9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9],
                         [pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, p9, p9, p9, pout9, pout9, pout9, pout9, pout9, pout9],
                         [pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, p9, p9, p9, pout9, pout9, pout9, pout9, pout9, pout9],
                         [pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, p9, p9, p9, pout9, pout9, pout9, pout9, pout9, pout9],
                         [pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, p9, p9, p9, pout9, pout9, pout9],
                         [pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, p9, p9, p9, pout9, pout9, pout9],
                         [pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, p9, p9, p9, pout9, pout9, pout9],
                         [pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, p9, p9, p9],
                         [pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, p9, p9, p9],
                         [pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, pout9, p9, p9, p9]])
        net3 = np.array([[p3, p3, p3, p3, p3, p3, p3, p3, p3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3],
                         [p3, p3, p3, p3, p3, p3, p3, p3, p3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3],
                         [p3, p3, p3, p3, p3, p3, p3, p3, p3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3],
                         [p3, p3, p3, p3, p3, p3, p3, p3, p3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3],
                         [p3, p3, p3, p3, p3, p3, p3, p3, p3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3],
                         [p3, p3, p3, p3, p3, p3, p3, p3, p3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3],
                         [p3, p3, p3, p3, p3, p3, p3, p3, p3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3],
                         [p3, p3, p3, p3, p3, p3, p3, p3, p3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3],
                         [p3, p3, p3, p3, p3, p3, p3, p3, p3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3],
                         [pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, p3, p3, p3, p3, p3, p3, p3, p3, p3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3],
                         [pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, p3, p3, p3, p3, p3, p3, p3, p3, p3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3],
                         [pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, p3, p3, p3, p3, p3, p3, p3, p3, p3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3],
                         [pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, p3, p3, p3, p3, p3, p3, p3, p3, p3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3],
                         [pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, p3, p3, p3, p3, p3, p3, p3, p3, p3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3],
                         [pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, p3, p3, p3, p3, p3, p3, p3, p3, p3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3],
                         [pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, p3, p3, p3, p3, p3, p3, p3, p3, p3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3],
                         [pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, p3, p3, p3, p3, p3, p3, p3, p3, p3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3],
                         [pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, p3, p3, p3, p3, p3, p3, p3, p3, p3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3],
                         [pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, p3, p3, p3, p3, p3, p3, p3, p3, p3],
                         [pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, p3, p3, p3, p3, p3, p3, p3, p3, p3],
                         [pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, p3, p3, p3, p3, p3, p3, p3, p3, p3],
                         [pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, p3, p3, p3, p3, p3, p3, p3, p3, p3],
                         [pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, p3, p3, p3, p3, p3, p3, p3, p3, p3],
                         [pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, p3, p3, p3, p3, p3, p3, p3, p3, p3],
                         [pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, p3, p3, p3, p3, p3, p3, p3, p3, p3],
                         [pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, p3, p3, p3, p3, p3, p3, p3, p3, p3],
                         [pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, pout3, p3, p3, p3, p3, p3, p3, p3, p3, p3]])
    
        if t>=t_start and  t < deltat1:
            return net27
        elif t>= deltat1 and t<2*deltat1:
            return net9
        elif t>=2*deltat1 and t < t_end:
            return net3
        else:
            print('Warning : t must be >=0 and <= 4*deltat1' +\
                  't is ', t)
            return np.eye(27)
    return block_mod_func
 


block_prob_mod_func = make_step_block_probs(deltat1, p27, p9, p3)

In [22]:
# create agents for the simlation
individuals = []

inter_tau = 2
activ_tau = 2

n_per_group = 3
n_groups = 27

for g in range(n_groups):

    individuals.extend([Individual(i, inter_distro_scale=inter_tau,
                                      activ_distro_scale=activ_tau,
                                      group=g) for i in range(g*n_per_group,(g+1)*n_per_group)])

In [23]:
# run simulation
sim = SynthTempNetwork(individuals=individuals, t_start=t_start, t_end=t_end,
                       next_event_method='block_probs_mod',
                       block_prob_mod_func=block_prob_mod_func)

print('running simulation')
sim.run(save_all_states=True, save_dt_states=True, verbose=False)

running simulation


In [24]:
net_heat = ContTempNetwork(source_nodes=sim.indiv_sources,
                        target_nodes=sim.indiv_targets,
                        starting_times=sim.start_times,
                        ending_times=sim.end_times,
                        merge_overlapping_events=True)

PID  2057156  :  merged  1976  events
PID  2057156  :  merged  0  events


In [25]:
net_heat.save('fig3_growing_network300_temporal_heat')